In [120]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

def setCookiesFromJson():
    with open('cookies.json', 'r', newline='') as inputdata:
        cookies = json.load(inputdata)
    for cookie in cookies: #works only after driver.get
        driver.add_cookie(cookie)
    driver.refresh() # to load cookies

service = Service(executable_path="chromedriver.exe")
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging']) #disable error logging
driver = webdriver.Chrome(service=service, options=chrome_options)

# driver.get("https://theprotocol.it/filtry/python;t/trainee,assistant,junior;p/zdalna;rw/praca/data-scientist-deep-learning-warszawa,oferta,8e0c0000-a5a4-c69a-3164-08dcbb7766ef?s=-1499548823&searchId=20a1f390-66d0-11ef-8a60-57a9d1202edd")
driver.get("https://theprotocol.it/filtry/python;t")
setCookiesFromJson()

In [121]:
action = webdriver.ActionChains(driver)

def waitAndHoverByClassName(name, timeout):
    element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CLASS_NAME, name)))
    # element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located('CLASS_NAME', name))
    print(element)
    # action.move_to_element(element) #move cursor
    # action.perform()

waitAndHoverByClassName("a4pzt2q", 5)

#o1onjy6t = container
#hulpcj = title
#a4pzt2q = offer class

<selenium.webdriver.remote.webelement.WebElement (session="2a8dae2197bd20f98918f5b2b4c9f47e", element="f.5CD989129CBF91D204CA2221ECE5797C.d.1AA5D053E7B25082B9CDC1A900B52730.e.321")>


In [122]:
offersContainer = driver.find_element(By.XPATH, '//*[@id="main-offers-listing"]/div[1]/div')
# offersContainer = driver.find_element("xpath", '//*[@id="main-offers-listing"]/div[1]/div') #also works

offers = offersContainer.find_elements(By.CLASS_NAME, 'a4pzt2q ') #
# offers = offersContainer.find_elements(By.CSS_SELECTOR, '#offer-title')

offers_urls = []

print('\t'+ str(len(offers)) + ' offers:')
# for offer in offers[0:1]:
for offer in offers:
    offers_urls.append(offer.get_property("href"))
    # print(offer_url)
    # driver.execute_script("window.open(\"" + offer_url + "\");")
    # time.sleep(0.5)
    # driver.switch_to.window(driver.window_handles[1])
    # driver.close()
    # driver.switch_to.window(driver.window_handles[0])
    # print(driver.current_url)

    # print(offer.text) #title
    # print(offer.get_property("innerHTML"))
    # print(offer.get_attribute('data-small')) #html attribute
    
for url in offers_urls:  
    print(url)

	50 offers:
https://theprotocol.it/szczegoly/praca/python-developer-warszawa,oferta,8e0c0000-a5a4-c69a-557e-08dcc052ad1b?searchId=961a5920-6bb0-11ef-b6ff-0d4dcd210125&s=-684638444
https://theprotocol.it/szczegoly/praca/python-developer-warszawa-pulawska-180,oferta,82b70000-8a10-6a4e-ddb2-08dcc1145276?searchId=961a5920-6bb0-11ef-b6ff-0d4dcd210125&s=-684638444
https://theprotocol.it/szczegoly/praca/python-developer-warszawa-krolewska-18,oferta,4f4d0000-ce9e-2e03-1ada-08dcb6023765?searchId=961a5920-6bb0-11ef-b6ff-0d4dcd210125&s=-684638444
https://theprotocol.it/szczegoly/praca/python-developer-opole-technologiczna-2,oferta,eef90000-e065-ee46-3495-08dccc023e9c?searchId=961a5920-6bb0-11ef-b6ff-0d4dcd210125&s=-684638444
https://theprotocol.it/szczegoly/praca/python-developer-krakow,oferta,5f840000-06f6-3a3a-94b3-08dcc76fb51e?searchId=961a5920-6bb0-11ef-b6ff-0d4dcd210125&s=-684638444
https://theprotocol.it/szczegoly/praca/test-automation-python-gdansk,oferta,c8350000-1514-9ec2-22eb-08dcb6e168